In [1]:
# Importing all the libraries needed
import numpy as np
import pandas as pd
import scipy 
import matplotlib as mplt
import os
from pydub import AudioSegment
from python_speech_features import mfcc, logfbank, ssc
import scipy.io.wavfile as wav

/home/alex/anaconda2/lib/python2.7/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
# Some variable and path initializations
os.chdir("/home/alex/Desktop/Bits,Coding and Extra/Bits_pilani_projects_and_competitions/Apogee_Project/apogee_amgc/genres1")
ori_folder_flag = 1
song_no = 0
df = pd.DataFrame(columns=['mfcc_feat','fbank_feat','ssc','classification'],index=range(0,1000))

In [3]:
# Getting all the genres
l = []
for dirpaths,dirnames,filenames in os.walk(os.getcwd()):
    l.append(dirnames)

In [ ]:
# Extracting Features of the songs

for i in l[0]:
    #for genre in genres1
    for x in range(100):
        # for song in batch of songs
        
        # Extracting the features
        (rate,sig) = wav.read(i+'/'+i+'.000'+"%02d"%x+'.wav')
        mfcc_feat = mfcc(sig,rate,nfft=551)
        fbank_feat = logfbank(sig,rate,nfft=551)
        sig_temp = np.reshape(sig, (sig.shape[0],1))
        ssc_var = ssc(sig_temp,rate,nfft=551)
        
        #Adding features to the pandas dataframe
        df.iloc[song_no][0] = mfcc_feat[0:2992,:]
        df.iloc[song_no][1] = fbank_feat[0:2992,:]
        df.iloc[song_no][2] = ssc_var[0:2992,:]
        df.iloc[song_no][3] = i
        
        # Incrementing the song number
        song_no+=1
        
    